# Imports and Constants

In [10]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from sklearn.pipeline import Pipeline

from savingOutputs import *
from loadData import *
from masks import *
from decoding import *
from plots import *
from utility import *
from metrics import *


SEED = 0
random.seed(SEED)
classes = ['Up', 'Down', 'Right', 'Left']
nb_runs = 12
length_one_modality = nb_runs * len(classes)
subjects_ids = range(1, 24)
n_subjects = len(subjects_ids)
n_individual_perms = 1000
labels = {'vis' : np.array(classes*nb_runs), 'aud' : np.array(classes*nb_runs)}
labels_same = np.array(classes*nb_runs)

within_modal_tasks_regions = [(["vis"], ["V5_L", "V5_R"]),
                              (["vis"], ["PT_L", "PT_R"]),
                              (["aud"], ["V5_L", "V5_R"]),
                              (["aud"], ["PT_L", "PT_R"])]

cross_modal_tasks_regions = [(["vis", "aud"], ["V5_L", "V5_R"]),
                             (["vis", "aud"], ["PT_L", "PT_R"])]

# Analyses

In [11]:
from_who = "our"   # 2 possibilities : "mohameds" or "our"
voxel_size = "3"
#radius_mask = "7"
radiuses = range(5,12)
maps_folder="brain_maps/"+from_who+"_maps"+"_"+voxel_size
#masks_folder="masks/"+from_who+"_masks"+"_"+voxel_size+"_radius="+radius_mask

scaler = sklearn.preprocessing.StandardScaler()
classifier = sklearn.svm.SVC(C = 1, kernel = "linear", random_state=SEED)
pipeline = Pipeline([('std_scaler', scaler), ('svm', classifier)])

In [ ]:
acc_group_combination = dict()
start_time = time.time()
for i, radius in enumerate(radiuses) :
    masks_folder="masks/"+from_who+"_masks"+"_"+str(voxel_size)+"_radius="+str(radius)
    maps_masked, masks_exist = load_full_data(subjects_ids, len(classes), nb_runs, maps_folder, masks_folder, is_from_mohamed=(from_who=="mohameds"))

    decoder = Decoder(n_perm=n_individual_perms, model=pipeline, n_classes=len(classes), n_splits=nb_runs, seed=SEED)
    decoder.set_masks_exist(masks_exist)

    # launch analyses
    confusion_matrixes_within = decoder.within_modality_decoding(maps_masked, labels, subjects_ids, within_modal_tasks_regions)
    group_confusion_matrixes_within = average_dicos(confusion_matrixes_within)

    confusion_matrixes_cross = decoder.cross_modality_decoding(maps_masked, labels, subjects_ids, cross_modal_tasks_regions)
    group_confusion_matrixes_cross = average_dicos(confusion_matrixes_cross)

    # saving results
    params = classifier.get_params()
    out_directory = "out/"+from_who+"_"+str(voxel_size)+"_radius="+str(radius)+"_"+str(classifier)+"/"
    create_directory(out_directory)

    save_dicts(out_directory + "confusion_matrixes_within.csv", confusion_matrixes_within,
               list(confusion_matrixes_within[0].keys()), subjects_ids)
    acc_within = compute_metric(out_directory, subjects_ids, {'name' : 'accuracy', 'function':accuracy}, "within", masks_exist, len(classes), ret = True)

    save_dicts(out_directory + "confusion_matrixes_cross.csv", confusion_matrixes_cross,
               list(confusion_matrixes_cross[0].keys()), subjects_ids)
    acc_cross = compute_metric(out_directory, subjects_ids, {'name' : 'cross', 'function':accuracy}, "within", masks_exist, len(classes), ret = True)

    within_modality_group_results = average_dicos(acc_within)
    save_dicts(out_directory + "group_scores_within.csv", [within_modality_group_results],
               list(within_modality_group_results.keys()), [0])
    acc_group_combination[radius] = within_modality_group_results

    cross_modality_group_results = average_dicos(acc_cross)
    save_dicts(out_directory + "group_scores_cross.csv", [cross_modality_group_results],
               list(cross_modality_group_results.keys()), [0])
    acc_group_combination[radius].update(cross_modality_group_results)


    duration = time.time()-start_time
    print(str(i+1)+"/"+str(len(radiuses))+" iterations done in "+str(duration)+" seconds")

1/7 iterations done in 41.28396248817444 seconds


# Plot scores depending on parameters

In [ ]:
import math
plt_directory = "plots/"+from_who+"_"+voxel_size+"_radiuses_"+str(classifier)+"/"
create_directory(plt_directory)
plotter = Plotter(plt_directory, subjects_ids)

#n_analyses = len(list(acc_group_combination["linear"][0].keys()))
#fig, axs = plt.subplots(nrows=math.ceil(n_analyses/2),ncols=2,figsize=(15,25))

for i, analysis in enumerate(acc_group_combination[5]):
    # faire le plot
    scores = [acc_group_combination[radius][analysis] for radius in radiuses]
    plt.plot(radiuses, scores)
    plt.axhline(0.25, color="gray", alpha=0.5)
    plotter.save(analysis+" - accuracy depending on ROI radius","","mean accuracy","radius [mm]",legend="out")

#plt.suptitle("Accuracy depending on C and kernel of SVM",fontsize=24)
#plotter.save(analysis,"","mean accuracy","log10(C)",legend="best")